# Imports

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd

# Definições

In [ ]:
# 1) Embeddings 
modelo_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True},
)

# 2) Carregar o índice salvo
vector_db = FAISS.load_local(
    "data/vector_db/v1_faiss_vector_db",
    embeddings=modelo_embedding,
    allow_dangerous_deserialization=True,  
)


# Separar informações comuns a capitulos

In [ ]:
docs = list(vector_db.docstore._dict.values())

linhas = []
for doc in docs:
    linhas.append({
        "texto": doc.page_content,
        **doc.metadata   # espalha as chaves do metadata como colunas
    })

# Separar artigos 0
df = pd.DataFrame(linhas)
df_art_0 = df[df['artigo']=='artigo_0.txt']
df_art_0.to_csv("data/processed/v1_artigos_0.csv", index=False)

# Função para obter o texto do artigo 0 correspondente
def get_art_0(law, title, chapter, df_art_0):
    cond = (
        (df_art_0['lei'] == law) &
        (df_art_0['titulo'] == title) &
        (df_art_0['capitulo'] == chapter)
    )
    resultados = df_art_0[cond]
    if resultados.shape[0] > 0:
        resultados = '\n'.join(resultados['texto'].tolist())
        return resultados

# Maneira como o contexto era passado na v1.1

In [61]:
K = 10
SEARCH_TYPE = "mmr"  # ou "similarity"
pergunta = "dispensa de licitação por valor, o que é?"

# Retriever antigo
retriever = vector_db.as_retriever(
    search_type=SEARCH_TYPE,
    search_kwargs={
        "k": K,
    }
)

contexto = retriever.invoke(pergunta)
contexto

[Document(id='100d4584-55dc-44ec-91ee-ac819b432c66', metadata={'lei': 'D10024', 'titulo': 'TITULO_0', 'capitulo': 'CAPITULO_IV', 'artigo': 'artigo_0.txt', 'path': 'D10024/TITULO_0/capitulos/CAPITULO_IV/artigos/artigo_0.txt', 'start_index': 0, 'chunk_idx': 0, 'orig_doc_id': 15}, page_content='DA CONDUÇÃO DO PROCESSO\nÓrgão ou entidade promotora da licitação'),
 Document(id='4bb69873-f161-4d23-9cd1-f67abfba9754', metadata={'lei': 'L14133', 'titulo': 'TITULO_II', 'capitulo': 'CAPITULO_VIII', 'artigo': 'artigo_75.txt', 'path': 'L14133/TITULO_II/capitulos/CAPITULO_VIII/artigos/artigo_75.txt', 'start_index': 0, 'chunk_idx': 0, 'orig_doc_id': 269}, page_content='Art. 75. É dispensável a licitação:\nI - para contratação que envolva valores inferiores a R$ 100.000,00 (cem mil reais), no caso de obras e serviços\nde engenharia ou de serviços de manutenção de veículos automotores; (Vide Decreto nº 10.922, de 2021)\n(Vigência) (Vide Decreto nº 11.317, de 2022) Vigência (Vide Decreto nº 11.871, de 

# Nova maneira
- filro para não buscar em informações comuns (ditos artigos 0)
- Separação e ordenação lógica dos resultados do retriever
- função para organizar o contexto

In [ ]:
# Novo retriever com filtro para não buscar em artigos 0
retriever = vector_db.as_retriever(
    search_type=SEARCH_TYPE,
    search_kwargs={
        "k": K,
        "filter": {
            "artigo": { "$nin": ['artigo_0.txt'] }
        }
    }
)

contexto = retriever.invoke(pergunta)

# separação dos resultados do novo retriever
linhas = []
for doc in contexto:
    linhas.append({
        "texto": doc.page_content,
        **doc.metadata
    })

# Ordenação dos resultados
df_resultados = pd.DataFrame(linhas).sort_values(
    ['lei', 'titulo', 'capitulo', 'artigo', 'chunk_idx']
).reset_index(drop=True)

print(df_resultados.shape)
df_resultados.head()

(10, 9)


,texto,lei,titulo,capitulo,artigo,path,start_index,chunk_idx,orig_doc_id
0,[[PARAGRAPH: §4]]\n§ 4º Não serão aceitos dois...,D10024,TITULO_0,CAPITULO_VIII,artigo_30.txt,D10024/TITULO_0/capitulos/CAPITULO_VIII/artigo...,862,1,41
1,Art. 50. A autoridade competente para homologa...,D10024,TITULO_0,CAPITULO_XVI,artigo_50.txt,D10024/TITULO_0/capitulos/CAPITULO_XVI/artigos...,0,0,67
2,"empresas, públicos ou privados, nacionais ou i...",L13709,TITULO_0,CAPITULO_IX,artigo_55-L.txt,L13709/TITULO_0/capitulos/CAPITULO_IX/artigos/...,832,1,134
3,"empresas de pequeno porte que, no ano-calendár...",L14133,TITULO_I,CAPITULO_I,artigo_4.txt,L14133/TITULO_I/capitulos/CAPITULO_I/artigos/a...,802,1,182
4,"Art. 12. No processo licitatório, observar-se-...",L14133,TITULO_II,CAPITULO_I,artigo_12.txt,L14133/TITULO_II/capitulos/CAPITULO_I/artigos/...,0,0,195


# Função para escrita lógica do contexto recuperado

In [67]:
def get_pos_processed_context(df_resultados, df_art_0):
    df_cap = df_resultados[['lei', 'titulo', 'capitulo']].drop_duplicates().reset_index(drop=True)
    context = ''
    for law in df_cap['lei'].unique():
        context += f'\n\n<LEI {law}>\n'
        df_law = df_cap[df_cap['lei']==law].copy()
        for title in df_law['titulo'].unique():
            art_0 = get_art_0(law, title, 'CAPITULO_0', df_art_0)
            if art_0:
                context += f'{art_0}\n'
            if title != 'TITULO_0':
                context += f'<TITULO: {title}>\n'
            df_title = df_law[df_law['titulo']==title].copy()
            for chapter in df_title['capitulo'].unique():
                art_0 = get_art_0(law, title, chapter, df_art_0)
                if art_0:
                    context += f'{art_0}\n'
                if chapter != 'CAPITULO_0':
                    context += f'<CAPITULO: {chapter}>\n'
                cond_lei = (df_resultados['lei']==law)
                cond_titulo = (df_resultados['titulo']==title)
                cond_capitulo = (df_resultados['capitulo']==chapter)
                mask = cond_lei & cond_titulo & cond_capitulo
                df_chapter = df_resultados[mask].copy()
                for artigo in df_chapter['artigo'].unique():
                    df_article = df_chapter[df_chapter['artigo']==artigo].copy()
                    context += f'<ARTIGO: {artigo.replace(".txt", "")}>\n'
                    for _, row in df_article.iterrows():
                        context += f"{row['texto']}\n"
                    context += f'</ARTIGO: {artigo.replace(".txt", "")}>\n'
                if chapter != 'CAPITULO_0':
                    context += f'</CAPITULO: {chapter}>\n'
            if title != 'TITULO_0':
                context += f'</TITULO: {title}>\n'
        context += f'</LEI {law}>\n'
    return context.replace('\n[[SECTION:', '[[SECTION:')

contexto_processado = get_pos_processed_context(df_resultados, df_art_0)
print(contexto_processado)



<LEI D10024>
Presidência da República
Secretaria-Geral
Subchefia para Assuntos Jurídicos
DECRETO Nº 10.024, DE 20 DE SETEMBRO DE 2019
Vigência
Regulamenta a licitação, na modalidade pregão, na
forma eletrônica, para a aquisição de bens e a
contratação de serviços comuns, incluídos os serviços
comuns de engenharia, e dispõe sobre o uso da
dispensa eletrônica, no âmbito da administração pública
federal.
O PRESIDENTE DA REPÚBLICA, no uso das atribuições que lhe confere o art. 84, caput, incisos II, IV e VI,
alínea “a”, da Constituição, e tendo em vista o disposto no art. 2º, § 1º, da Lei nº 10.520, de 17 de julho de 2002, e na
Lei nº 8.666, de 21 de junho de 1993,
DECRETA:
DA ABERTURA DA SESSÃO PÚBLICA E DO ENVIO DE LANCES
Horário de abertura
<CAPITULO: CAPITULO_VIII>
<ARTIGO: artigo_30>
[[PARAGRAPH: §4]]
§ 4º Não serão aceitos dois ou mais lances iguais e prevalecerá aquele que for recebido e registrado primeiro.
[[PARAGRAPH: §5]]
§ 5º Durante a sessão pública, os licitantes serão info